In [ ]:
from  eanalytics_api_py import Conn, eaload
import time

gridpool_name = "dem"
datacenter = "com"
website_name = "demo-fr"
api_key = "" # api token here


date_from = '09/01/2020 00:00:00'
date_to = '09/02/2020 00:00:00'
pattern = '%m/%d/%Y %H:%M:%S'
epoch_from = int(time.mktime(time.strptime(date_from, pattern)))
epoch_to = int(time.mktime(time.strptime(date_to, pattern)))

conn = Conn(
        gridpool_name=gridpool_name,
        datacenter='com',
        api_key=api_key,
        print_log = True
)

query = """
GET {{
    TIMERANGE {{ {epoch_from} {epoch_to} }}
    READERS {{ 
        ea:clickview@{website_name} AS clickview
        ea:pageview@{website_name} AS pageview
    }}
    GROUPS{{
    visit WITH pageview IF {{visit.first.pageview.timestamp > pageview.timestamp + mins(30)}}
    }}
    JOINS {{
        clickview WITH visit IF {{
            clickview.timestamp >= visit.first.pageview.timestamp 
            && clickview.timestamp <= visit.last.pageview.timestamp
        }} AS join1
    }}
    OUTPUTS_ROW (join1) {{
        join1.clickview.timestamp,
        join1.clickview.uid,
        join1.clickview.type,
        join1.clickview.channel.odmedia,
        join1.clickview.channel.odata.ope.name,
        join1.clickview.channel.odata.publisher.name
    }}
}};""".format(
    website_name=website_name,
    epoch_from=epoch_from,
    epoch_to=epoch_to
)



path2file = conn.download_edw(
    query=query,
    output_filename = 'edw_{grid}_{website}_{_from}_{_to}_click_during_visit.csv.gzip'.format(
        grid=gridpool_name,
        website=website_name,
        _from=epoch_from,
        _to=epoch_to
    ),
    override_file=False,
)

df = eaload.generic.csv_files_2_df(path2file)
display(df.info(), df.head())